In [31]:
DL = '/Users/damiansp/Downloads'

In [32]:
!rm -rf $DL/*

In [33]:
from datetime import datetime, timedelta
import os

import numpy as np
import pandas as pd
import xlrd

In [34]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ADBE', 'ADYEY', 'AMD', 'AMGN', 'AMZN',
    'ANET', 'ASML', 'AXON', 'BROS', 'BYND', 'CHWY', 'CRWD', 'DDOG',
    'DOCS', 'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT', 'GH', 'GMED',
    'GOOG', 'IDXX', 'INTG', 'ISRG', 'JD', 'LRCX', 'LSPD', 'LULU', 'MDB',
    'MELI', 'MNST', 'MSFT', 'MTCH', 'NVCR', 'NVDA', 'ODFL', 'OKTA',
    'OPEN', 'PAC', 'PANW', 'PAYC', 'PINS', 'PYPL', 'QDEL', 'RBLX',
    'RDFN', 'RGEN', 'RIVN', 'ROKU', 'RVLV', 'SE', 'SHOP', 'SNOW',
    'TASK', 'TDOC', 'TEAM', 'TREX', 'TTD', 'TTWO', 'TWLO', 'TWTR',
    'TYL', 'UPST', 'VEEV', 'VLD', 'VRTX', 'XPEV', 'ZEN', 'ZM']
LINGERERS = [
    'AMEH', 'AMRK', 'BYRN', 'CIZN', 'CTSH', 'CUBE', 'EP', 'EXR', 'FCPT', 
    'FCUV', 'GBOX', 'GME', 'INTZ', 'KREF', 'LFMD', 'MCVT', 'MNMD',
    'OMQS', 'PETV', 'POWW', 'PRPH', 'RA', 'RVP', 'VIRT', 'WAVD', 'WKHS',
    'ZYXI']
BEST_WEIGHTED = [
    'ABST', 'APT', 'BKT', 'CEF', 'CFFN', 'CZFS', 'FCAP', 'FCN', 'FNV',
    'GCBC', 'HMY', 'IEI', 'MICS', 'MIY', 'MUJ', 'NAZ', 'NLY', 'NPV',
    'NXP', 'SOBR']
SYMBOLS = sorted(list(set(SYMBOLS + LINGERERS + BEST_WEIGHTED)))

In [35]:
!ls $DL

In [36]:
#!head -5 $DL/Positions.csv

In [37]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [38]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [39]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ABST
ADBE


### E*Trade

In [40]:
TODAY = datetime.now().date()
TODAY
DONGMEI = False

In [41]:
df = pd.read_csv(
    f'{DL}/Positions.csv', index_col=0, skiprows=1
)[['Market Value']].rename(columns={'Market Value': 'ET'})
df.index = map(lambda x: x.split()[0], df.index)
df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
df.head()

,ET
AAPL,1353.8700000000001
ABMD,1227.65
ABNB,305.25
ADBE,1138.24
ADYEY,3766.71


In [42]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/2022-04-29.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    et_df.ET = et_df.ET.apply(
        lambda x: round(float(x)) if x != '--' else x)
et_df.head()

,ET
AAPL,1354
ABMD,1228
ABNB,305
ADBE,1138
ADYEY,3767


In [43]:
#et_df.drop(index=['98986CONT'], inplace=True)
#et_df.head()

In [44]:
et_df.loc[et_df.ET == '--']
#et_df.loc['NVCR', 'ET'] = 79.24

,ET


In [45]:
check_symbols(et_df)

All clear


In [46]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,1354
ABMD,1228
ABNB,305
ABST,0
ADBE,1138


### Fidelity

In [47]:
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Sep-23-2022.csv'

In [48]:
!ls $DL

Portfolio_Positions_Sep-23-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_23.xls


In [49]:
#df_raw = pd.read_csv(f'{DL}/{filename.replace("-16-", "-17-")}')
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,775.32,$1.00,$0.00,$775.32,NaN,NaN,NaN,NaN,2.32%,NaN,NaN,Cash
1,218520300,ROLLOVER IRA,ABMD,ABIOMED INC COM USD0.01,4.00,$245.53,-$1.73,$982.12,-$6.92,-0.70%,-$36.57,-3.59%,2.93%,$1018.69,$254.67,Cash
2,218520300,ROLLOVER IRA,ABNB,AIRBNB INC COM CL A,2.00,$101.75,-$1.97,$203.50,-$3.94,-1.90%,-$82.00,-28.73%,0.61%,$285.50,$142.75,Cash
3,218520300,ROLLOVER IRA,ADYEY,ADYEN NV UNSPON ADS EACH REP 0.01 ORD SHS,13.00,$12.19,-$0.319,$158.47,-$4.15,-2.56%,-$56.22,-26.19%,0.47%,$214.69,$16.51,Cash
4,218520300,ROLLOVER IRA,AMD,ADVANCED MICRO DEVICES INC,9.00,$67.96,-$1.54,$611.64,-$13.86,-2.22%,-$235.41,-27.80%,1.83%,$847.05,$94.12,Cash


In [50]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,ROLLOVER IRA,ABMD,982
2,ROLLOVER IRA,ABNB,204
3,ROLLOVER IRA,ADYEY,158
4,ROLLOVER IRA,AMD,612
5,ROLLOVER IRA,AMZN,228


In [51]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
ABMD,982
ABNB,204
ADYEY,158
AMD,612
AMZN,228


In [52]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [53]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,1354,0,6017,0
ABMD,1228,982,3192,0
ABNB,305,204,509,0
ABST,0,0,0,0
ADBE,1138,0,0,0


### TD Ameritrade

In [54]:
#!cat $DL/Positions_damiansp_Stocks_2022_08_09.xls

In [55]:
today = str(TODAY).replace('-', '_')
path = f'{DL}/Positions_damiansp_Stocks_{today}.xls'
#tomorrow = str(TODAY + timedelta(1)).replace('-', '_')
#path = f'{DL}/Positions_damiansp_Stocks_{tomorrow}.xls'

In [56]:
# Stock (default) view
#td = pd.read_excel(, engine='xlrd')
def parse_file(path):
    inds = []
    data = []
    is_header = True
    with open(path, 'r') as f:
        for line in f:
            if not is_header:
                if line.startswith('"'):
                    try:
                        columns = line.split('\t')
                        symbol = columns[0][1:-1]
                        value = float(columns[5][1:-1])
                        inds.append(symbol)
                        data.append(value)
                    except IndexError:
                        print('Cannot parse:', line)
            if line.startswith('Symbol'):
                is_header = False
    return pd.DataFrame({'TD': data}, index=inds)

In [57]:
td = parse_file(path)
td.head()

,TD
ADBE,284.57
AMEH,41.09
AMRK,503.12
APT,35.91
BYND,15.54


In [58]:
out = pd.concat([out, td], axis=1).fillna(0).round().astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA,TD
AAPL,1354,0,6017,0,0
ABMD,1228,982,3192,0,0
ABNB,305,204,509,0,0
ABST,0,0,0,0,0
ADBE,1138,0,0,0,285


In [59]:
out.to_csv(f'{DL}/clean.csv')

In [60]:
!open $DL/clean.csv

In [31]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [10]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


In [11]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

.DS_Store
Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [12]:
check_symbols(mw)

All clear


In [13]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [14]:
mw.to_csv(f'{DL}/mw.csv')

In [47]:
!open $DL/mw.csv

The file /Users/damiansp/Downloads/mw.csv does not exist.


In [ ]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [ ]:
!rm $DL/*